In [1]:
!wget https://https://github.com/xaage/text_mining/blob/main/dataset/tripadvisor_reviews.tsv

In [2]:
# Instalamos nltk
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')
# Instalamos spacy y uno de sus modelos
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm
# Instalamos textacy
!pip install textacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 16.2MB 29.9MB/s 
  Create

# Ejercicio 2 

El objetivo de este ejercicio es comprobar los conocimientos adquiridos por el alumno en temas relacionados a las transformación de textos utilizando la técnica de TFIDF.

# Apartado 1 
Preprocesar los textos del corpus con las funciones de preprocesado creadas en el ejercicio 1 (quitar palabras vacías, quitar símbolos de puntuación, lematizar con spacy) 

In [3]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from wordcloud import WordCloud
import string 
import re 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from tqdm.autonotebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
# Cargamos la base de datos
import pandas as pd
dataset = pd.read_csv("tripadvisor_reviews.tsv", sep = "\t")

In [5]:
# Número de documentos duplicados
print("Existen {} reviews duplicadas".format(np.sum(dataset.duplicated(subset=["Review"]))))

Existen 0 reviews duplicadas


In [6]:
!pip install spacy
!python -m spacy download en_core_web_sm # para contenido en inglés

2021-07-12 07:30:48.520046: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 13.6MB 219kB/s 
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
#  Preprocesar los textos del corpus con las funciones de preprocesado creadas en el ejercicio 1 
# (quitar palabras vacías, quitar símbolos de puntuación, lematizar con spacy)

In [8]:
# Diseñamos funciones para la limpieza de los textos

# Eliminar espacios
def eliminar_espacios(text): 
    return  " ".join(text.split()) 

# Quitar palabras vacías
from nltk.corpus import stopwords
def quitar_stopwords(tokens):
    stop_words = set(stopwords.words('english')) 
    filtered_sentence = [w for w in tokens if not w in stop_words] 
    return filtered_sentence

# Quitar símbolos de puntuación 
def quitar_puntuacion(tokens):
    words=[word for word in tokens if word.isalnum()]
    return words

# Lematizar con spacy
import en_core_web_sm 
nlp = en_core_web_sm.load(disable=['parser', 'ner']) 
def lematizar(tokens):
    sentence = " ".join(tokens)
    mytokens = nlp(sentence)
    # Lematizamos los tokens y los convertimos  a minusculas
    mytokens = [ word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    # Extraemos el text en una string
    return " ".join(mytokens)

# Tokenizador utilizando nltk 
def tokenization(text): 
  tokens = word_tokenize(text)
  return tokens 

In [9]:
# Podemos empezar con la normalización

# Quitar Espacios
dataset["normaliza"] = dataset["Review"].progress_apply(lambda x: eliminar_espacios(x))
# Tokenizar
dataset["normaliza"] = dataset["normaliza"].progress_apply(lambda x: tokenization(x))
# Quitar Stopwords
dataset["normaliza"] = dataset["normaliza"].progress_apply(lambda x: quitar_stopwords(x))
# Quitar puntuación
dataset["normaliza"] = dataset["normaliza"].progress_apply(lambda x: quitar_puntuacion(x))
# Stemming
dataset["normaliza"] = dataset["normaliza"].progress_apply(lambda x: lematizar(x))

dataset["normaliza"]

0        nice hotel expensive parking get good deal sta...
1        ok nothing special charge diamond member hilto...
2        nice room experience hotel monaco seattle good...
3        unique great stay wonderful time hotel monaco ...
4        great stay great stay go seahawk game awesome ...
                               ...                        
20486    well keep secret 3rd time stay charm ca beat t...
20487    great location price view hotel great quick pl...
20488    ok look nice modern outside desk staff particu...
20489    hotel theft ruin vacation hotel open sept 17 2...
20490    people talk ca believe excellent rating hotel ...
Name: normaliza, Length: 20491, dtype: object

# Apartado 2 
Utilizar la función de scikit-learn para realizar la transformación a TF-IDF del texto considerando que:

- Se consideren unigramas, bigramas y trigramas.
- No se tengan en cuenta los elementos que aparezcan en menos del 5% de los textos.
- Se utilice el tokenizador incorporado en Scikit-Learn. 
- Haya un máximo de 200 características. 

En scikit-learn podemos utilizar distintas funciones para obtener el vocabulario de un corpus de documentos. Ambas están presentes dentro del módulo feature_extraction.text y son CountVectorizer y TfidfVectorizer. Vamos a crear un objeto con cada una de esas clases para introducir nuestro corpus y extraer el vocabulario

# Vectorización

Una vez hemos limpiado el texto, vamos a extraer características utilizando TFIDFVectorizer, ademas de utilizar el tokenizador incorporado en Scikit-Learn.

Queremos que: 
- Se consideren unigramas, bigramas y trigramas
- No se tengan en cuenta los elementos que aparezcan en menos del 5% de los textos 
- Se utilice el tokenizador incorporado en Scikit-Learn
- Haya un máximo de 200 características



In [10]:
# Utilizamos TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Transformamos el dataset limpiado anteriormente
# Scikit-learn no utiliza stopwords
# Pero se utilice el tokenizador incorporado en Scikit-Learn
vectorizador = TfidfVectorizer(min_df=0.05, ngram_range=(1,3), tokenizer= None, max_features=200)
vector_data = vectorizador.fit(dataset["normaliza"])

In [11]:
# Mostramos el diccionario que mapea las palabras con TfidfVectorizer
print("\n\n TF-IDF VECTORIZER")
print("Obtenemos un dictionario que mapea las palabras con su posición en el vector del vocabulario")
print(vector_data.vocabulary_)
print("Obtenemos el vocabulario en si mismo como una lista")
print(vector_data.get_feature_names)
print("Numero de características:")
print(len(vector_data.get_feature_names()))



 TF-IDF VECTORIZER
Obtenemos un dictionario que mapea las palabras con su posición en el vector del vocabulario
{'nice': 113, 'hotel': 84, 'expensive': 55, 'get': 70, 'good': 73, 'stay': 165, 'arrive': 5, 'evening': 52, 'take': 171, 'review': 144, 'check': 28, 'easy': 46, 'little': 95, 'view': 186, 'room': 146, 'clean': 31, 'size': 154, 'bed': 14, 'comfortable': 36, 'high': 81, 'like': 94, 'hear': 78, 'night': 114, 'morning': 109, 'door': 43, 'open': 120, 'people': 124, 'long': 99, 'location': 98, 'great': 74, 'overall': 122, 'experience': 56, 'pay': 123, 'stay hotel': 166, 'room clean': 147, 'nothing': 116, 'charge': 27, 'book': 18, 'suite': 169, 'extra': 57, 'bathroom': 11, 'standard': 162, 'reservation': 140, 'desk': 40, 'say': 148, 'thing': 173, 'tv': 182, 'tell': 172, 'free': 66, 'breakfast': 19, 'kid': 90, 'call': 24, 'day': 38, 'offer': 118, 'guest': 76, 'ask': 6, 'provide': 132, 'hard': 77, 'trip': 180, 'staff': 160, 'helpful': 80, 'block': 17, 'away': 8, 'know': 91, 'help': 

Count_data

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# Utilizamos count_vectorizer
count_vectorizer = CountVectorizer()
# Hacemos fit 
count_data = count_vectorizer.fit(dataset["normaliza"])

In [13]:
# Mostramos el diccionario con CountVectorizer
print("COUNT VECTORIZER")
print("Obtenemos un dictionario que mapea las palabras con su posición en el vector del vocabulario")
print(count_data.vocabulary_)
print("Obtenemos el vocabulario en si mismo como una lista")
print(count_data.get_feature_names())
print("Numero de características:")
print(len(count_data.get_feature_names()))

COUNT VECTORIZER
Obtenemos un dictionario que mapea las palabras con su posición en el vector del vocabulario
{'nice': 26229, 'hotel': 19249, 'expensive': 14775, 'parking': 28149, 'get': 17091, 'good': 17410, 'deal': 11302, 'stay': 36248, 'anniversary': 3295, 'arrive': 3912, 'late': 22271, 'evening': 14385, 'take': 37460, 'advice': 2295, 'previous': 29984, 'review': 32310, 'valet': 40716, 'check': 8448, 'quick': 30702, 'easy': 13297, 'little': 22882, 'disappointed': 12261, 'view': 41068, 'room': 32730, 'clean': 8999, 'size': 34857, 'bed': 5396, 'comfortable': 9461, 'wake': 41382, 'stiff': 36383, 'neck': 25999, 'high': 18803, 'pillow': 28996, 'soundproof': 35554, 'like': 22712, 'hear': 18521, 'music': 25710, 'night': 26282, 'morning': 25404, 'loud': 23203, 'bang': 4852, 'door': 12777, 'open': 27177, 'closing': 9130, 'people': 28565, 'talk': 37479, 'hallway': 18166, 'maybe': 24245, 'noisy': 26425, 'neighbor': 26074, 'aveda': 4430, 'bath': 5112, 'product': 30173, 'goldfish': 17379, 'touch